<a href="https://colab.research.google.com/github/eshmaapps/UIUtils/blob/master/dresser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a notebook that builds a model to identify a traditional dresser against a contemporary or industrial style dresser.

In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
print(tf.__version__)

2.6.0


In [19]:
import pathlib
dataset_url = "https://storage.googleapis.com/furndata/data.zip"

data_dir = tf.keras.utils.get_file(origin=dataset_url, 
                                   fname='data', 
                                   untar=True)
data_dir = pathlib.Path(data_dir)

87498752/87488946 [==============================] - 1s 0us/step


In [ ]:
import glob
for name in data_dir.glob("*"):
    print(name)

In [20]:
!pwd
!ls -ltr /root/.keras/datasets/

/content
total 85440
-rw-r--r-- 1 root root 87488946 Sep 27 16:21 data.tar.gz


In [ ]:
# First run unzip data.tar.gz
!cd /root/.keras/datasets/
!unzip /root/.keras/datasets/data.tar.gz -d ./
print(len(list(data_dir.glob('*/*'))))


In [ ]:
traditional = list(data_dir.glob('*/*'))

In [25]:
!ls -tlr /root/.keras/datasets/
!cp /root/.keras/datasets/data.tar.gz /content/
!unzip /content/data.tar.gz

total 85440
-rw-r--r-- 1 root root 87488946 Sep 27 16:21 data.tar.gz
Archive:  /content/data.tar.gz
replace data/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [35]:
!ls -ltr /content/data/data

total 20
drwxr-xr-x 2 root root  4096 Sep 24 18:08 contemporary
drwxr-xr-x 2 root root 12288 Sep 24 18:09 traditional
drwxr-xr-x 2 root root  4096 Sep 24 18:10 industrial


In [36]:
data_dir = "/content/data/data"
data_dir = pathlib.Path(data_dir)

print(len(list(data_dir.glob('*/*'))))


686


In [37]:
batch_size = 32
img_height = 180
img_width = 180

In [38]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 683 files belonging to 3 classes.
Using 547 files for training.


In [39]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 683 files belonging to 3 classes.
Using 136 files for validation.


In [40]:
class_names = train_ds.class_names
print(class_names)

['contemporary', 'industrial', 'traditional']


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [41]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

In [42]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [43]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [44]:
IMG_SIZE = 180

resize_and_rescale = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
])
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [45]:
num_classes = 3

model = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [46]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [47]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10
)

Epoch 1/10
18/18 [==============================] - 26s 1s/step - loss: 1.2582 - accuracy: 0.3583 - val_loss: 1.0809 - val_accuracy: 0.4044
Epoch 2/10
18/18 [==============================] - 20s 1s/step - loss: 1.0639 - accuracy: 0.4314 - val_loss: 1.0524 - val_accuracy: 0.4044
Epoch 3/10
18/18 [==============================] - 20s 1s/step - loss: 1.0161 - accuracy: 0.4333 - val_loss: 1.0050 - val_accuracy: 0.4118
Epoch 4/10
18/18 [==============================] - 21s 1s/step - loss: 0.9503 - accuracy: 0.4863 - val_loss: 0.9776 - val_accuracy: 0.5294
Epoch 5/10
18/18 [==============================] - 21s 1s/step - loss: 0.9248 - accuracy: 0.5594 - val_loss: 0.9838 - val_accuracy: 0.4265
Epoch 6/10
18/18 [==============================] - 21s 1s/step - loss: 0.8222 - accuracy: 0.6417 - val_loss: 1.0015 - val_accuracy: 0.5588
Epoch 7/10
18/18 [==============================] - 21s 1s/step - loss: 0.7897 - accuracy: 0.6399 - val_loss: 0.9696 - val_accuracy: 0.5294
Epoch 8/10
18/18 [==

In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10
)

In [48]:
checkpoint_path = "cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=1,
  callbacks=[cp_callback]
)
# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

18/18 [==============================] - 21s 1s/step - loss: 0.3817 - accuracy: 0.8519 - val_loss: 1.0566 - val_accuracy: 0.6029

Epoch 00001: saving model to cp.ckpt


In [49]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
#loss, acc = model.evaluate(test_images, test_labels, verbose=2)


In [54]:
#Import libraries
#import pandas
import pandas as pd
#importing numpy
import numpy as np
#importing tensorflow
import tensorflow as tf
#importing keras from tensorflow
from tensorflow import keras
from keras import layers
# importing Sequential from keras
from tensorflow.keras.models import Sequential
#importing Dense and Conv2D layers from keras
from tensorflow.keras.layers import Dense,Conv2D

def build_model(hp):
    # create model object
    model = Sequential([
    #adding first convolutional layer    
    layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        # adding filter size or kernel size
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        #activation function
        activation='relu',
        input_shape=(28,28,1)),
    #layers.MaxPool2D(),
    # adding second convolutional layer 
    layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        #adding filter size or kernel size
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        #activation function
        activation='relu'
    ),
    #layers.MaxPool2D(),
    layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_3_filter', min_value=32, max_value=64, step=16),
        #adding filter size or kernel size
        kernel_size=hp.Choice('conv_3_kernel', values = [3,5]),
        #activation function
        activation='relu'
    ),# adding flatten layer    
    #layers.MaxPool2D(),
    layers.Flatten(),
    # adding dense layer    
    layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    # output layer    
    layers.Dense(3, activation='softmax')
    ])
    #compilation of model
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [51]:
train_labels = np.concatenate([y for x, y in train_ds], axis=0)
val_labels = np.concatenate([y for x, y in val_ds], axis=0)


In [57]:
train_labels.shape

(547,)

In [52]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 97 kB 2.9 MB/s 


In [55]:
#importing random search

from kerastuner import RandomSearch
#creating randomsearch object
tuner = RandomSearch(build_model,
                    objective='val_accuracy',
                    max_trials = 5)
# search best parameter
tuner.search(train_ds,train_labels,epochs=3,validation_data=(train_ds,train_labels))


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |80                |?                 
conv_1_kernel     |5                 |?                 
conv_2_filter     |48                |?                 
conv_2_kernel     |3                 |?                 
conv_3_filter     |32                |?                 
conv_3_kernel     |3                 |?                 
dense_1_units     |112               |?                 
learning_rate     |0.001             |?                 



ValueError: ignored